## Stub Plans 

### Links Úteis

- Documentação dos plan stubs: https://blueskyproject.io/bluesky/main/plans.html#stub-plans

- Documentação da customização de planos com per_step: https://blueskyproject.io/bluesky/main/plans.html#customize-step-scans-with-per-step

- Documentação da utilização de metadados: https://blueskyproject.io/bluesky/main/metadata.html

- Documentação dos preprocessors: https://blueskyproject.io/bluesky/main/plans.html#plan-preprocessors

In [ ]:
%matplotlib qt
from time import time
from bluesky import RunEngine
from bluesky.plans import count
from bluesky.callbacks import LiveTable
from bluesky.plan_stubs import abs_set, rel_set, mv, mvr, \
    stage, unstage, stop, pause, deferred_pause, sleep, \
    checkpoint, clear_checkpoint, repeat, trigger_and_read, \
    open_run, close_run
from ophyd import Device
from ophyd.sim import noisy_det, motor1, motor2
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

#### Getting values from ophyd objects

In [ ]:
print(f"Read: {motor1.read()}")
print()
print(f"Configuration: {motor1.read_configuration()}")
print()
print(f"Get Device: {motor1.get()}")
print()
print(f"Get Readback: {motor1.readback.get()}")

## mv and abs_set

In [ ]:
#help(abs_set)
#help(mv)

In [ ]:
RE(mv(motor1, 0))
motor1.delay = 2
print(f"Before: {motor1.read()}")
RE(abs_set(motor1, 15))
print(f"After: {motor1.read()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 0))
motor1.delay = 2
print(f"Before: {motor1.read()}")
RE(abs_set(motor1, 15, wait=True))
print(f"After: {motor1.read()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 0, motor2, 0))
motor1.delay = 2
motor2.delay = 4
print(f"Before : {motor1.readback.get()}")
print(f"Before motor2: {motor1.readback.get()}")
start = time()
RE(mv(motor1, 4, motor2, 15))
end = time()
print(f"After motor1: {motor1.readback.get()}")
print(f"After motor2: {motor2.readback.get()}")
print(f"Duration: {end - start}")
motor1.delay = 0
motor2.delay = 0

## mvr and rel_set

In [ ]:
#help(mvr)
#help(rel_set)

In [ ]:
RE(mv(motor1, 14))
motor1.delay = 2
print(f"Before: {motor1.readback.get()}")
RE(rel_set(motor1, 15))
print(f"After: {motor1.readback.get()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 14))
motor1.delay = 2
print(f"Before: {motor1.readback.get()}")
RE(mvr(motor1, 15))
print(f"After +15: {motor1.readback.get()}")
RE(mvr(motor1, -10))
print(f"After -10: {motor1.readback.get()}")
RE(mvr(motor1, 2.5))
print(f"After +2.5: {motor1.readback.get()}")
motor1.delay = 0

## sleep and stop

In [ ]:
#help(sleep)
#help(stop)

In [ ]:
class StopDevice(Device):

    def stop(self):
        super().stop()
        print("This Device has been stopped")

In [ ]:
stop_device = StopDevice(name="stop_device")
RE(stop(stop_device))

In [ ]:
start = time()
RE(sleep(1))
end = time()
print(f"Duration: {end - start}")

## stage and unstage

In [ ]:
#help(stage)
#help(unstage)

In [ ]:
class StagedDevice(Device):

    def stage(self):
        super().stage()
        print("This Device is being armed")

    def unstage(self):
        super().unstage()
        print("This Device is being restored to a safe idle state")

In [ ]:
exampleDevice = StagedDevice(name="staged_device")

def stage_unstage_plan():
    yield from stage(exampleDevice)
    
    # Custom plan
    print("Custom plan")
    
    yield from unstage(exampleDevice)

RE(stage_unstage_plan())

## Controlling the Run Engine 

In [ ]:
#help(checkpoint)
#help(clear_checkpoint)
#help(pause)
#help(deferred_pause)
#help(open_run)
#help(close_run)

### Pause

In [ ]:
def pause_plan():
    yield from open_run()
    yield from mv(motor1, 5)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from pause()
    
    yield from mv(motor1, 4)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from mv(motor1, 14)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from close_run()

In [ ]:
RE(pause_plan(), LiveTable(["noisy_det"]))

In [ ]:
print(RE.state)
RE.resume()

### Deferred Pause

In [ ]:
def deferred_pause_plan():
    yield from open_run()
    yield from mv(motor1, 5)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from deferred_pause()
    
    yield from mv(motor1, 4)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from mv(motor1, 14)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from close_run()

In [ ]:
RE(deferred_pause_plan(), LiveTable(["noisy_det"]))

In [ ]:
print(RE.state)
RE.resume()

### clear_checkpoint

In [ ]:
def clear_checkpoint_plan():
    yield from open_run()
    yield from mv(motor1, 5)
    yield from trigger_and_read([noisy_det])
    yield from clear_checkpoint()
    
    yield from pause()
    
    yield from mv(motor1, 4)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from mv(motor1, 14)
    yield from trigger_and_read([noisy_det])
    yield from checkpoint()
    
    yield from close_run()

In [ ]:
RE(clear_checkpoint_plan(), LiveTable(["noisy_det"]))

In [ ]:
RE.state

## repeat

In [ ]:
help(repeat)

In [ ]:
def repeat_count():
    yield from count([noisy_det], num=5, delay=0.25)
RE(repeat(repeat_count, num=3, delay=3), LiveTable(["noisy_det"]))

Fly Scan


Plans for asynchronous acquisition:

monitor: Asynchronously monitor for new values and emit Event documents.

unmonitor: Stop monitoring.

kickoff: Kickoff one fly-scanning device.

complete: Tell a flyable, 'stop collecting, whenever you are ready'.

collect: Collect data cached by one or more fly-scanning devices and emit documents.


Standard plans: per_step


Combinations of the above that are often convenient:

trigger_and_read(devices[, name])
	

Trigger and read a list of detectors and bundle readings into one Event.

one_1d_step(detectors, motor, step[, ...])
	

Inner loop of a 1D step scan

one_nd_step(detectors, step, pos_cache[, ...])
	

Inner loop of an N-dimensional step scan

one_shot(detectors[, take_reading])
	

Inner loop of a count.

move_per_step(step, pos_cache)
	

Inner loop of an N-dimensional step scan without any readings